In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
# Verificar si CUDA está disponible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# Definir el modelo y moverlo a la GPU
class ComplexModel(nn.Module):
    def __init__(self):
        super(ComplexModel, self).__init__()
        self.fc1 = nn.Linear(2, 64) # Capa de entrada con 2 neuronas y 64 neuronas de salida
        self.fc2 = nn.Linear(64, 128) # Capa oculta con 64 neuronas de entrada y 128 neuronas de salida
        self.fc3 = nn.Linear(128, 64) # Capa oculta con 128 neuronas de entrada y 64 neuronas de salida
        self.fc4 = nn.Linear(64, 1) # Capa de salida con 64 neuronas de entrada y 1 neurona de salida
        self.relu = nn.ReLU() # Función de activación ReLU

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        return x

In [4]:
model = ComplexModel()
model.to(device) # Mover el modelo a la GPU


ComplexModel(
  (fc1): Linear(in_features=2, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

In [5]:
# Generar datos y etiquetas
numbers = np.arange(256)
integers = np.arange(-4, 5)
pairs = np.array(np.meshgrid(numbers, integers)).T.reshape(-1, 2)

def get_num_ones(number):
    binary = format(number, '08b')
    num_ones = sum([int(bit) for bit in binary])
    return num_ones

labels = []
for pair in pairs:
    number = pair[0]
    integer = pair[1]
    num_ones = get_num_ones(number)
    if (integer == 0) or num_ones == 4 - num_ones:
        labels.append(1)
    else:
        labels.append(0)

pairs = torch.tensor(pairs, dtype=torch.float32).to(device)
labels = torch.tensor(labels, dtype=torch.float32).to(device)


In [6]:
# Crear función de pérdida y optimizador
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Entrenar el modelo
epochs = 100000000
for epoch in range(epochs):
    # Pasar datos de entrada a través del modelo y obtener predicciones
    outputs = model(pairs)
    
    # Calcular la pérdida
    loss = criterion(outputs, labels.unsqueeze(1))
    
    # Reiniciar gradientes y realizar retropropagación
    optimizer.zero_grad()
    loss.backward()
    
    # Actualizar los pesos del modelo
    optimizer.step()
    
    # Imprimir la pérdida en cada epoch
    if (epoch + 1) % 1000 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [1000/100000000], Loss: 0.2454
Epoch [2000/100000000], Loss: 0.2386
Epoch [3000/100000000], Loss: 0.2431
Epoch [4000/100000000], Loss: 0.2392
Epoch [5000/100000000], Loss: 0.2343
Epoch [6000/100000000], Loss: 0.3832
Epoch [7000/100000000], Loss: 0.5904
Epoch [8000/100000000], Loss: 0.2231
Epoch [9000/100000000], Loss: 0.2191
Epoch [10000/100000000], Loss: 0.2155
Epoch [11000/100000000], Loss: 0.2120
Epoch [12000/100000000], Loss: 0.2092
Epoch [13000/100000000], Loss: 0.2736
Epoch [14000/100000000], Loss: 0.2159
Epoch [15000/100000000], Loss: 0.2122
Epoch [16000/100000000], Loss: 0.2080
Epoch [17000/100000000], Loss: 0.2051
Epoch [18000/100000000], Loss: 0.2027
Epoch [19000/100000000], Loss: 0.2009
Epoch [20000/100000000], Loss: 0.1984
Epoch [21000/100000000], Loss: 0.3215
Epoch [22000/100000000], Loss: 0.2227
Epoch [23000/100000000], Loss: 0.2104
Epoch [24000/100000000], Loss: 0.2073
Epoch [25000/100000000], Loss: 0.2063
Epoch [26000/100000000], Loss: 0.2040
Epoch [27000/10000000

KeyboardInterrupt: 

In [8]:
# Evaluar el modelo
with torch.no_grad():
    # Pasar datos de entrada a través del modelo y obtener predicciones
    outputs = model(pairs)
    
    # Redondear las predicciones a 0 o 1
    predictions = torch.round(torch.sigmoid(outputs))
    
    # Calcular la precisión
    accuracy = (predictions == labels.unsqueeze(1)).sum().item() / labels.size(0) * 100
    print(accuracy)

96.22395833333334


In [9]:
# Guardar los pesos del modelo en un archivo .pth
torch.save(model.state_dict(), 'indicatrizE.pth')
print("Los pesos del modelo se han guardado")

Los pesos del modelo se han guardado
